Business.json cleaing

In [7]:
import pandas as pd

df = pd.read_json('yelp_academic_dataset_business.json', encoding='utf-8', lines=True)

In [9]:
df.describe()

,latitude,longitude,stars,review_count,is_open
count,150346.000000,150346.000000,150346.000000,150346.000000,150346.00000
mean,36.671150,-89.357339,3.596724,44.866561,0.79615
std,5.872759,14.918502,0.974421,121.120136,0.40286
min,27.555127,-120.095137,1.000000,5.000000,0.00000
25%,32.187293,-90.357810,3.000000,8.000000,1.00000
50%,38.777413,-86.121179,3.500000,15.000000,1.00000
75%,39.954036,-75.421542,4.500000,37.000000,1.00000
max,53.679197,-73.200457,5.000000,7568.000000,1.00000


In [1]:
import pandas as pd
import json

# Load business.json
def load_business_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)

# Load data
business_df = load_business_json("/Users/vinayakgoswamy/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json")

# Drop 'postal_code' if present
if 'postal_code' in business_df.columns:
    business_df.drop(columns=['postal_code'], inplace=True)

# Drop rows missing review count
business_df = business_df.dropna(subset=['name', 'stars', 'review_count'])

# Show average number of reviews
avg_reviews = business_df['review_count'].mean()
print(f"Average number of reviews across businesses: {avg_reviews:.2f}")


Average number of reviews across businesses: 44.87


In [2]:
# Filter out businesses with fewer than 15 reviews
filtered_business_df = business_df[business_df['review_count'] >= 5]

# Reset index for cleanliness
filtered_business_df.reset_index(drop=True, inplace=True)




In [5]:
# Example path
output_path = "/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_business.csv"

filtered_business_df.to_csv(output_path, index=False)


In [ ]:
csv_string = filtered_business_df.to_csv(index=False)
print(csv_string[:1000])  # Show first 1000 characters so you can copy and preview


business_id,name,address,city,state,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}"
tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'CoatCheck': 'False', 'RestaurantsTakeOut': 'False', 'RestaurantsDelivery': 'False', 'Caters': 'False', 'WiFi': ""u'no'"", 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'WheelchairAccessible': 'True', 'HappyHour': 'False', 'OutdoorSeating': 'False', 'HasTV': 'False',

In [6]:
# Load the CSV back into a DataFrame to verify it
test_df = pd.read_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_business.csv")
print(test_df.head())  # Show the first few rows to check the content


              business_id                      name  \
0  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
1  tUFrWirKiKi_TAnsVWINQQ                    Target   
2  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
3  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
4  n_0UpQx1hsNbnPUSlodU8w           Famous Footwear   

                                      address          city state   latitude  \
0             87 Grasso Plaza Shopping Center        Affton    MO  38.551126   
1                        5255 E Broadway Blvd        Tucson    AZ  32.223236   
2                                 935 Race St  Philadelphia    PA  39.955505   
3                               101 Walnut St    Green Lane    PA  40.338183   
4  8522 Eager Road, Dierbergs Brentwood Point     Brentwood    MO  38.627695   

    longitude  stars  review_count  is_open  \
0  -90.335695    3.0            15        1   
1 -110.880452    3.5            22        0   
2  -75.155564    4.0            80        1   


review.json Cleaning

In [8]:
import pandas as pd
import json

# Load cleaned business IDs (from your already cleaned business_df)
business_df = pd.read_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_business.csv")
valid_business_ids = set(business_df['business_id'])

# Load review.json in chunks to avoid memory issues
review_data = []
with open("/Users/vinayakgoswamy/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json", "r", encoding="utf-8") as f:
    for line in f:
        review = json.loads(line)
        # Filter directly while reading
        if review['business_id'] in valid_business_ids:
            review_data.append(review)

# Convert to DataFrame
review_df = pd.DataFrame(review_data)
print("Total filtered reviews loaded:", len(review_df))


Total filtered reviews loaded: 6641275


In [9]:
# Drop rows with missing critical fields
review_df.dropna(subset=['review_id', 'user_id', 'business_id', 'stars', 'text'], inplace=True)


# Drop duplicates
review_df.drop_duplicates(subset='review_id', inplace=True)

# Remove reviews with very short text
review_df['text'] = review_df['text'].astype(str)
review_df = review_df[review_df['text'].str.strip().str.len() >= 30]


print("Cleaned review count:", len(review_df))


Cleaned review count: 6640275


In [10]:
review_df.to_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_review.csv", index=False)


In [11]:
review_df_further = review_df[~((review_df['useful'] == 0) & 
                        (review_df['funny'] == 0) & 
                        (review_df['cool'] == 0))]

print("Cleaned review count:", len(review_df_further))


review_df_further.to_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/further_cleaned_review.csv", index=False)


Cleaned review count: 3355929


In [12]:
test_df = pd.read_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_review.csv", nrows=50)
print(test_df.head()) 

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0    3.0       0      0     0   
1    5.0       1      0     1   
2    3.0       0      0     0   
3    5.0       1      0     1   
4    4.0       1      0     1   

                                                text                 date  
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year...  2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm...  2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delici

Checkin. json

In [13]:
import pandas as pd
import json

# Load checkin data
with open("/Users/vinayakgoswamy/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_checkin.json", "r") as f:
    checkin_data = [json.loads(line) for line in f]

checkin_df = pd.DataFrame(checkin_data)

# Drop rows with missing business_id
checkin_df = checkin_df.dropna(subset=["business_id"])


# Filter checkin data to keep only those with valid business_id
filtered_checkin_df = checkin_df[checkin_df["business_id"].isin(valid_business_ids)]



# Save to CSV (or preview in memory)
filtered_checkin_df.to_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_checkin.csv", index=False)

# Preview
filtered_checkin_df.head()


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."
5,--8IbOsAAxjKRoYsBFL-PA,"2015-06-06 01:03:19, 2015-07-29 16:50:58, 2015..."


Tips.json

In [14]:
import pandas as pd
import json

# Load tips.json
with open("/Users/vinayakgoswamy/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_tip.json", "r") as f:
    tip_data = [json.loads(line) for line in f]

tip_df = pd.DataFrame(tip_data)

# Drop rows with missing critical fields
tip_df = tip_df.dropna(subset=[ "date", "business_id", "user_id"])

# Filter by valid business_ids

tip_df = tip_df[tip_df["business_id"].isin(valid_business_ids)]

tip_df = tip_df[tip_df["compliment_count"] > 0]


# Save cleaned tips
tip_df.to_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_tip.csv", index=False)

# Preview
tip_df.head()


,user_id,business_id,text,date,compliment_count
24,tA1U-XSh9woo73eQmWGyAQ,xHwvbm1SJwtaZtOZzFQcmQ,If you haven't been here in a good while or i...,2016-06-11 23:18:23,1
52,nhXyjGOCfi-EW09bJqfAzQ,1Efad30BdOeqqjX2d6P4sw,You MUST have the popover breakfast!!,2015-05-23 13:33:04,1
210,RNupfU2QClRq_lRR3mEgrQ,5kWhgFvaf9_zeyOfO-2NxQ,"Try the calamari, the chicken Marsala was grea...",2013-02-15 02:29:41,1
332,alUlVVMx9NtfrvK4xAQy2w,C7WUvmGAz_t4FE_ycOPGUA,Try the in house eye of round at the deli. Sli...,2015-09-06 23:00:23,1
355,tslp3KJf3tATFHJnFzeyVA,UakVMT3xrpbFB2pHdxPjnw,This is the undisputed breakfast champion on D...,2012-10-13 14:02:43,1


users.json

In [1]:
import json
import pandas as pd

filtered_users = []

with open("/Users/vinayakgoswamy/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_user.json", "r") as f:
    for line in f:
        user = json.loads(line)

        # Check for missing user_id or name
        if not user.get("user_id") or not user.get("name"):
            continue

        # Check review count
        if user.get("review_count", 0) == 0:
            continue

        

        # All conditions passed
        filtered_users.append(user)



# Convert to DataFrame
user_df = pd.DataFrame(filtered_users)


print("Cleaned user count:", len(user_df))





Cleaned user count: 1987843


In [2]:
compliment_cols = [col for col in user_df.columns if col.startswith('compliment_')]
user_df['compliment_sum'] = user_df[compliment_cols].sum(axis=1)
user_df = user_df[user_df['compliment_sum'] > 0]

# Drop the temporary sum column
user_df = user_df.drop(columns=['compliment_sum'])
print("Cleaned user count:", len(user_df))


# (Optional) Save to CSV for future use
user_df.to_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/cleaned_users.csv", index=False)

# Preview
user_df.head()

Cleaned user count: 621529


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [3]:
# 3. Drop users with 0 fans
further_user_df = user_df[user_df['fans'] > 0]
print("Cleaned user count:", len(further_user_df))
further_user_df.to_csv("/Users/vinayakgoswamy/Desktop/DSM Final Project/further_cleaned_users.csv", index=False)



Cleaned user count: 314058
